In [ ]:
import datetime
import pandas as pd

### Variables

In [ ]:
DATAFILE = 'malta.csv'
CITIES_HOME = ['Katowice', 'Kraków', 'Poznań', 'Warszawa']
CITIES_DEST = ['Valletta']
NIGHTS_RANGE = range(6, 10)
PRICE_MAX_ROUNDTRIP = 700
HOLYDAYS = [
    datetime.date(2025, 11, 8), datetime.date(2025, 11, 9), # weekend
    datetime.date(2025, 11, 15), datetime.date(2025, 11, 16), # weekend
    datetime.date(2025, 11, 22), datetime.date(2025, 11, 23), # weekend
    datetime.date(2025, 11, 10), datetime.date(2025, 11, 11), # independence day
]
WORKDAY_HOUR_AFTER = 14
ALLOW_STOPS = True

### Data

In [ ]:
data = pd.read_csv(DATAFILE)
data['departure'] = pd.to_datetime(data['departure'])
data['arrival'] = pd.to_datetime(data['arrival'])
data['duration'] = pd.to_timedelta(data['duration'])

### Filter and pair up

In [ ]:
# filter flights based on price, stops and time (no flight before specific hour on workdays)
data = data[data['price'] < (PRICE_MAX_ROUNDTRIP - data['price'].min())]
if not ALLOW_STOPS: data = data[data['stops'].isna()]
data = data[data['departure'].dt.date.isin(HOLYDAYS) | (data['departure'].dt.hour >= WORKDAY_HOUR_AFTER)]

# combine into pairs
pairs = []
outbound_flights = data[data['start'].isin(CITIES_HOME) & data['end'].isin(CITIES_DEST)]
inbound_flights = data[data['start'].isin(CITIES_DEST) & data['end'].isin(CITIES_HOME)]
for _, outbound in outbound_flights.iterrows():
    for _, inbound in inbound_flights.iterrows():
        nights = (inbound['departure'].date() - outbound['departure'].date()).days
        if nights not in NIGHTS_RANGE: continue
        price = outbound['price'] + inbound['price']
        if price > PRICE_MAX_ROUNDTRIP: continue
        # free days
        free_days = sum(outbound['arrival'].date() < d < inbound['departure'].date() for d in HOLYDAYS)
        if outbound['arrival'].date() in HOLYDAYS and outbound['arrival'].hour < 15: free_days += 1
        if inbound['departure'].date() in HOLYDAYS and inbound['departure'].hour >= 15: free_days += 1
        pairs.append({
            **{f'out_{k}': v for k, v in outbound.to_dict().items()},
            **{f'in_{k}': v for k, v in inbound.to_dict().items()},
            'nights': nights,
            'free_days': free_days,
            'total_price': price
        })
pairs = pd.DataFrame(pairs)

### Display

In [ ]:
format_airport = lambda row, prefix: f"{row[f'{prefix}_start']} -> {row[f'{prefix}_stops']} -> {row[f'{prefix}_end']}" \
    if pd.notna(row[f'{prefix}_stops']) \
    else f"{row[f'{prefix}_start']} -> {row[f'{prefix}_end']}"
format_date = lambda row, prefix: row[f'{prefix}_departure'].strftime('%a %d.%m.%Y')
format_time = lambda row, prefix: f"{row[f'{prefix}_departure'].strftime('%H:%M')}-{row[f'{prefix}_arrival'].strftime('%H:%M')}"
format_duration = lambda row, prefix: f"{row[f'{prefix}_duration'].seconds // 3600}h {(row[f'{prefix}_duration'].seconds // 60) % 60}m"

flights = pd.DataFrame()
flights['out_airport'] = pairs.apply(lambda row: format_airport(row, 'out'), axis=1)
flights['out_date'] = pairs.apply(lambda row: format_date(row, 'out'), axis=1)
flights['out_time'] = pairs.apply(lambda row: format_time(row, 'out') + ' (' + format_duration(row, 'out') + ')', axis=1)
flights['nights'] = pairs['nights']
flights['free_days'] = pairs['free_days']
flights['in_airport'] = pairs.apply(lambda row: format_airport(row, 'in'), axis=1)
flights['in_date'] = pairs.apply(lambda row: format_date(row, 'in'), axis=1)
flights['in_time'] = pairs.apply(lambda row: format_time(row, 'in') + ' (' + format_duration(row, 'in') + ')', axis=1)
flights['total_price'] = pairs['total_price']

flights = flights.sort_values(by=['free_days', 'total_price'], ascending=[False, True])
flights.reset_index(drop=True, inplace=True)
flights

In [ ]:
flights[flights['free_days'] >= 5]